In [1]:
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [2]:
from datasets import *

In [3]:
(x,y)=DATA['iris']

In [5]:
def normalize_cols(m):
    col_max=m.max(axis=0)
    col_min=m.min(axis=0)
    return (m-col_min)/(col_max-col_min)

In [6]:
x=normalize_cols(x)

In [7]:
(n,m)=x.shape

In [8]:
import matplotlib
classes=np.unique(y)
colors=matplotlib.pyplot.cm.gist_rainbow(np.linspace(0,1,len(classes)))
#colors=matplotlib.pyplot.cm.rainbow(np.linspace(0,1,len(classes)))
cm = matplotlib.colors.ListedColormap(colors)
axiscolors=matplotlib.pyplot.cm.rainbow(np.linspace(0,1,m))

In [9]:
def Arcviz(x,alpha,pm):
	#alpha
	m=len(x)
	theta=np.zeros(m)
	
	for i in range(m):
		theta[i]=alpha[pm[i]]*x[pm[i]]+alpha[pm[(i+1)%m]]*(1.0-x[pm[i]])
		
	anchor=np.array([np.cos(theta),np.sin(theta)])
	px=np.dot(anchor,x)/np.sum(x)
	return px

In [10]:
def circleradviz(X,a):
	#p=sum 
	(n,m)=X.shape
	Y=np.zeros((n,2))
	alpha=a
	order=np.argsort(a)
	for i in range(n):
		Y[i]=Arcviz(X[i],alpha,order)
	return Y

In [11]:
def Classifier(x,y):
	clf=LDA()
	clf.fit(x,y)
	ac=clf.score(x,y)
	#ac=silhouette(x, y, metric='sqeuclidean')
	return ac

In [12]:
STEP=0.15

In [13]:
def animate_arcviz(t):
    A1=(t*np.log(2.0))*STEP%(2*np.pi)
    A2=(t*np.log(3.0))*STEP%(2*np.pi)
    A3=(t*np.log(5.0))*STEP%(2*np.pi)
    A4=(t*np.log(7.0))*STEP%(2*np.pi)
    
    Alpha=np.array([A1,A2,A3,A4])
    
    m=len(Alpha)
    anchor=np.zeros((m,2))
    for i in range(m):
        anchor[i,0]=np.cos(Alpha[i])
        anchor[i,1]=np.sin(Alpha[i])
        
    Y=circleradviz(x,Alpha)
    #draw Radviz
    plt.figure(figsize=(6,6))
    plt.scatter(Y[:,0],Y[:,1],c=y,cmap=cm,marker='.',s=10)
    #draw unit circle
    t = np.linspace(0,2*np.pi, 100)
    plt.plot(np.cos(t),np.sin(t),alpha=0.95)
    #draw alpha point
    (n,m)=x.shape
    al=Alpha
    
    pm=np.argsort(al)
    for i in range(m+1):
        t1=al[pm[i%m]]
        t2=al[pm[(i+1)%m]]
        if (t1>t2):
            t2=t2+2*np.pi
        t=np.linspace(t1,t2,50)
        plt.plot(np.cos(t),np.sin(t),color=axiscolors[pm[i%m]])
        plt.scatter(np.cos(t1),np.sin(t1),color="red",alpha=0.95);
        plt.text((1+0.1)*np.cos(0.5*(t1+t2)),(1+0.1)*np.sin(0.5*(t1+t2)),'A'+str(pm[i%m]+1),color="black")
    
    ac=Classifier(Y,y)
    plt.text(-1,1,str(ac*100)[:5]+'%')
    plt.xlim(-1.2,1.2)
    plt.ylim(-1.2,1.2)
    for i in range(m):
        #np.linspace(V[i,0],V[i,1], 30)
        #plt.scatter(anchor[i,0],anchor[i,1],color=axiscolors[i],size=al[i]*10,alpha=0.95)
        plt.scatter(anchor[i,0],anchor[i,1],color=axiscolors[i],alpha=0.95)
        #plt.text((1+0.1)*anchor[i,0],(1+0.1)*anchor[i,1],'A'+str(i+1),color="black")
    plt.show()
   

    
    
    return ac

In [14]:
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
play = widgets.Play(
    value=1,
    min=1,
    max=100,
    step=1,
    description="Press play"
)
slider = widgets.FloatSlider()
widgets.jslink((play, 'value'), (slider, 'value'))

In [16]:
p=interactive(animate_arcviz,t=play)
widgets.VBox([slider,p])

VBox(children=(FloatSlider(value=1.0), interactive(children=(Play(value=1, description='Press play', min=1), Output()), _dom_classes=('widget-interact',))))